# Background
Gradient descent is the optimization technique used to optimize the parameters of a system. I say system because gradient descent is a general purpose technique that can be used to optimize the parameters that influence any output. The assumption here is that the computations are differentiable. Back propagation lies at the heart of gradient descent. It is used to calculate the gradients (derivatives) of the parameters. The mechanical steps performed to compute the gradients of the parameters is back propagation. In this article we will see simple examples of how back propagation computes the gradients. 

### Simple Neuron
We will be looking at Simple Neural Networks. Neural networks take in one or multidimensional input to produce one or multidimensional output. To do that they are composed of a bunch of neurons that are parallel to each other forming a layer. Multiple layer can be chained together to create "deep" neural networks.

![Simple Neuron](simple_neuron.png){#fig-neuron}

### Forward Pass

@fig-neuron illustrates a simple neuron. It takes in an input, performs some computation with its weight and produces an output. Application of a non-linear activation function is not explicitly shown but can be thought of as part of the computation. Non-linear activation functions play a crucial role in giving Neural Networks the flexibility to solve non-linear problems. I have omitted the bias term for simplicity which can be thought of as the offset that gets added on before the activation function. The input could be multidimensional in which case the weight would also be multidimensional. The weights multiply the inputs and the results get summed up (the sum would apply to multiple weight input pairs and the bias term). The summed value is passed through the non-linear activation function producing the final output.  

### Derivatives

We can clearly see that since we are using weight to multiply the input it affects the output. How much does the weight influence the output can be calculated by taking the derivate of the output with respect to the weight. Multidimensional derivatives are called gradients. This article uses derivatives and gradients interchangeably. If there are multiple weights (as is the case when the inputs are multidimensional), since each weight contributes (by multiplying) to the output, each weight will have a derivative with respect to the output. The derivative is the degree of influence the computation has in producing the output. When we want to influence the output in a certain direction we can use the derivative to adjust the weight.

### Backward pass

At a high level inputs are fed into the network which produces output. We can think of the outputs as the predictions of the model. In supervised learning we know the desired output of the each input. We can compare the output of the model and desired outputs (labels) to create a error value. We use a *loss* function for that. If we aggregate all the errors we get a scalar value which quantifies the mistakes the model made in the predictions. The wights played a role in creating the outputs thus the error. We can propagate the information captured int the error back to the weights giving the weights an opportunity to update in the desired direction of the output. The derivatives of the weights provide the direction in which the weights are to be adjusted. 

![Chain rule](simple_fb_pass.png){#fig-chain-rule}

As there can be arbitrary number of neurons in a layer and arbitrary number of layers we need a systematic way to calculate the gridents of each weight. Back propagation is this systematic method that computes the gradients of the error with respect to each weight. In a deep neural network the output of previous layer are fed as input to the subsequent layer till the final error is computed. Since output of a computation is being fed as input into another computation which is a compositie function. Chain rule in calculus gives us a way to calculate the gradients for composite functions. The chain rule in calculus states that if you have a composite function, the derivative of the output with respect to an input variable is the product of the derivatives of the intermediate steps. 

### Chain rule

@fig-chain-rule illustrates forward and backward pass of a neural network that has only two neurons in two layers. In the forward pass the input (x) is influenced by w1 to produce x'. x' in turn is influenced by w2 to produce output (o). Calculating the derivative of w2 with respect to o can be done in a single step. Derivative of w1 with respect to o needs to take into consideration the intermediate values computed along the way. The right thing to do here is to multiply the intermediate derivatives. 

### Conclusion 

That is pretty much all there is to back propogation. The opertation to compute individual derivatives can be straight forward. The complexity rises from having to calculate and keep track of intermediate derivatives. Imagine hundreds of these layers and their associated computations. Frameworks like pytorch make is incredibly simple by abstracting away the complexity. Manually computing the gradients layer by layer would surely be tedious not to mention error prone. In this example we only looked at scalar valued output. In practical terms inputs are vectors and layers of neurons then turn into matrices. Vectors stacked on top of eachother become matrices so the computation end up becoming matrix matrix operations. Frameworks like pytorch also take this into consideration making gradient computation a simple `.backward()` method call.  

### Postscript

[Andrej Karpathy](https://www.youtube.com/watch?v=VMj-3S1tku0) has a video on creating micrograd; it is a simple codebase that has a simple class that does a bunch of operations. Each operation also has an associated method that accumulates the derivative. I would highly recommend watching the video. Below are some snippets of code on my attempt at implementing the class. It only shows the add method for brevity. When two `Value`s are added the result output could further go on to participate in other computations. Instead of worrying about the global operation we only worry about the local derivative (which is 1 in our example of addition) multiplied with whatever the derivative of the output is with respect to the global output. As long as we systematically follow this starting from the end (back to front - crucial to propagate gradients correctly) we will have the correct derivaties. 

In [6]:
class Value:
    """Class that implements basic autograd functionality similar to pytorch"""

    def __init__(self, data, child=(), label="", op=""):
        self.data = data
        self._prev = set(child)
        self.label = label
        self._op = op
        self.grad = 0 # Initially gradient is zero
        self._backward = lambda : None

    def __add__(self, other):
        if isinstance(other, (int, float)): other = Value(other)
        out = Value(self.data + other.data, child=(self, other), op="+")
        def backward():
            self.grad += out.grad # We are omitting multiplying with local derivative which is * 1 in addition
            other.grad += out.grad
        out._backward = backward
        return out

#### Resources:
- [My implementation of micrograd](https://github.com/ishandahal/micrograd)
- [Andrej's micrograd repo](https://github.com/karpathy/micrograd)
- [Drawing made possible by Excalidraw](https://excalidraw.com/)